##### Initialization: definition of robots and coordinates

In [ ]:
import numpy as np
from UAV import *
import random
import matplotlib.pyplot as plt   


n_robots = 3#int(input("number of robots: "))
n_components = 2
platoon = []


# initialization of n robots
for i in range(n_robots):
    if i==0:
        i_robot = Robot("op_" + str(i),0,0,0)
    else:
        i_robot = Robot("op_" + str(i),random.uniform(0, 10.0),random.uniform(0, 10.0),0)
    platoon.append(i_robot)

coordinates = [[],[],[]]

for rob in platoon:
    coordinates = np.append(coordinates,rob.get_coords(),axis=1)

#TRUE COORDINATES [REALITY: THEY ARE UNKNOWN]
S = coordinates[0:2,:]

print(S)

##### Definition of the (squared) distance matrix

In [ ]:
DM2 = d_matrix2(platoon) #squared distance matrix -> for formulas
print(DM2)

#double check the distances
print('\n',DM2 - DM_from_S(S))

##### S* estimation through EigenValue Decomposition

In [ ]:
S_star = EVD(DM2,n_components)
print(S_star)

##### Removal of the ambiguity generated by distances in an Universal Euclidean transformation

In [ ]:
#Translation removal
S_star = match_anchor(S,S_star,verbose=1)
print(S_star)

The new coordinates may be affected by rotation and flip ambiguities. Thus we must solve the problem with further analysis

##### Identification of rotation ambiguity

DM' : a new acquisition for the distances

We now introduce a displacement for the anchor, in order to build a new (squared) distance matrix.

In reality what we do is applying a displacement to the anchor and we get a new distance matrix

In [ ]:
deltaX1 = 0.3
deltaY1 = 0.1

DeltaS_prime = np.array([[0 for i in range(n_robots)] for j in range(2)],dtype=float)
DeltaS_prime[:,0] = [deltaX1,deltaY1]

S_prime = S + DeltaS_prime
print(S_prime)

DM_prime = DM_from_S(S_prime,verbose=1)

Estimation of the rotation angle $\theta_r$

In [ ]:
"""theta1a, theta1b = estimate_theta2(DM2,DM_prime,S_star,[deltaX1,deltaY1],index=1,verbose=1)
print(theta1a)
print(theta1b)

theta2a, theta2b = estimate_theta2(DM2,DM_prime,S_star,[deltaX1,deltaY1],index=2,verbose=1)
print(theta2a)
print(theta2b)"""

print("------------------------------------")
theta3a, theta3b = estimate_theta3(DM2,DM_prime,S_star,[deltaX1,deltaY1],index=1,verbose=1)
print(theta3a)
print(theta3b)

theta4a, theta4b = estimate_theta3(DM2,DM_prime,S_star,[deltaX1,deltaY1],index=2,verbose=1)
print(theta4a)
print(theta4b)

In [ ]:
theta_r   = estimate_theta(DM2,DM_prime,S_star,[deltaX1,deltaY1],verbose=1)
theta_r_a = estimate_theta(DM2,DM_prime,S_star,[deltaX1,deltaY1],approx = 1,verbose=1)

S** : estimated-rotated coordinates

In [ ]:
S_star2 = rotateMatrix(theta_r)@S_star
S_star2a = rotateMatrix(theta_r_a)@S_star

##### Identification of flip ambiguity

We have to check if there is any flip ambiguity. To do so, we have to move the anchor one more time

In [ ]:
deltaX2 = 0.7
deltaY2 = 0.4

DeltaS_prime2 = np.array([[0 for i in range(n_robots)] for j in range(2)],dtype=float)
DeltaS_prime2[:,0] = [deltaX2,deltaY2]

S_prime2 = S + DeltaS_prime2
print(S_prime)

DM_prime2 = DM_from_S(S_prime2,verbose=1)

Now we estimate a second rotation angle $\theta_{r2}$: if = 0, no flip ambiguity

In [ ]:
theta_r2 = estimate_theta(DM2,DM_prime2,S_star2,[deltaX2,deltaY2],approx=2,verbose=0)
print("theta_r2 : ", theta_r2)


if (theta_r2 != 0):
  F = np.array([[-1,0],[0,1]])
    
  theta_r3 = estimate_theta(DM2,DM_prime,F@S_star,[deltaX1,deltaY1],verbose=0)
  theta_r3_a = estimate_theta(DM2,DM_prime,F@S_star,[deltaX1,deltaY1],approx=1,verbose=0)

  print("Wrong estimated angle:",theta_r)
  print("True  estimated angle:", theta_r3)

  S_star2 = rotateMatrix(theta_r3)@F@S_star
  S_star2a = rotateMatrix(theta_r3_a)@F@S_star


plot_points(S,S_star  = S_star,
              S_star2 = S_star2,
              S_star2a = S_star2a
            )